In [5]:
from ultralytics import YOLO
import math
import torch

# Load the trained models
model_1 = YOLO('best_model1.pt')
model_2 = YOLO('best_model2.pt')
model_3 = YOLO('best_model3.pt')
model_4 = YOLO('best_model4.pt')
model_5 = YOLO('best_model5.pt')

# Function to calculate the model weight (alpha) based on the error rate
def calculate_model_weight(error_rate):
    return 0.5 * math.log((1 - error_rate) / error_rate)

# Example: Replace these with your actual error rates for each model
error_rate_1 = 0.2  # Example error rate for best_model1.pt
error_rate_2 = 0.2 # Example error rate for best_model2.pt
error_rate_3 = 0.2 # Example error rate for best_model3.pt
error_rate_4 = 0.2 # Example error rate for best_model4.pt
error_rate_5 = 0.2 # Example error rate for best_model5.pt

# Calculate weights (alphas) for each model
alpha_1 = calculate_model_weight(error_rate_1)
alpha_2 = calculate_model_weight(error_rate_2)
alpha_3 = calculate_model_weight(error_rate_3)
alpha_4 = calculate_model_weight(error_rate_4)
alpha_5 = calculate_model_weight(error_rate_5)

# Ensemble method to combine predictions using AdaBoost-inspired weights
def ensemble_adaboost(image_path, models, alphas):
    all_boxes = []
    all_scores = []
    all_classes = []
    
    for model, alpha in zip(models, alphas):
        results = model.predict(image_path, conf=0.25)  # Adjust confidence threshold if needed
        
        for result in results:
            boxes = result.boxes
            all_boxes.append(boxes.xyxy)
            all_scores.append(boxes.conf * alpha)  # Apply alpha weight to confidence scores
            all_classes.append(boxes.cls)
    
    if all_boxes:
        # Concatenate all predictions
        boxes_tensor = torch.cat(all_boxes)
        scores_tensor = torch.cat(all_scores)
        classes_tensor = torch.cat(all_classes)

        # Perform NMS
        nms_indices = torch.ops.torchvision.nms(boxes_tensor, scores_tensor, 0.5)
        
        # Create final predictions
        final_boxes = boxes_tensor[nms_indices]
        final_scores = scores_tensor[nms_indices]
        final_classes = classes_tensor[nms_indices]
        
        return list(zip(final_boxes, final_scores, final_classes))
    
    return []

# Combine predictions using the ensemble method
models = [model_1, model_2, model_3, model_4, model_5]
alphas = [alpha_1, alpha_2, alpha_3, alpha_4, alpha_5]

# Example image to test the ensemble model
image_path = 'javanicus.jpeg'
final_predictions = ensemble_adaboost(image_path, models, alphas)

# Print or visualize the final predictions
for i, (box, score, cls) in enumerate(final_predictions):
    print(f"Detection {i+1}:")
    print(f"  Box: {box.tolist()}")
    print(f"  Score: {score.item():.4f}")
    print(f"  Class: {cls.item()}")


image 1/1 /home/fnilvu/Documents/Project Medaka/API Ikan/javanicus.jpeg: 352x640 1 O.javanicus, 25.6ms
Speed: 1.7ms preprocess, 25.6ms inference, 1.3ms postprocess per image at shape (1, 3, 352, 640)

image 1/1 /home/fnilvu/Documents/Project Medaka/API Ikan/javanicus.jpeg: 352x640 1 O.javanicus, 25.7ms
Speed: 1.5ms preprocess, 25.7ms inference, 1.8ms postprocess per image at shape (1, 3, 352, 640)

image 1/1 /home/fnilvu/Documents/Project Medaka/API Ikan/javanicus.jpeg: 352x640 1 O.javanicus, 25.9ms
Speed: 1.6ms preprocess, 25.9ms inference, 2.1ms postprocess per image at shape (1, 3, 352, 640)

image 1/1 /home/fnilvu/Documents/Project Medaka/API Ikan/javanicus.jpeg: 352x640 1 O.javanicus, 25.6ms
Speed: 2.5ms preprocess, 25.6ms inference, 1.1ms postprocess per image at shape (1, 3, 352, 640)

image 1/1 /home/fnilvu/Documents/Project Medaka/API Ikan/javanicus.jpeg: 352x640 1 O.javanicus, 25.6ms
Speed: 1.6ms preprocess, 25.6ms inference, 1.1ms postprocess per image at shape (1, 3, 352, 

In [6]:
import torch

# Save the ensemble configuration
def save_ensemble_config(model_paths, alphas, file_path='ensemble_model_config.pt'):
    """
    Save the ensemble model configuration to a file.

    Args:
    - model_paths (list of str): List of paths to the model files.
    - alphas (list of float): List of alpha weights for each model.
    - file_path (str): Path where the ensemble configuration will be saved.
    """
    ensemble_config = {
        'model_paths': model_paths,
        'alphas': alphas
    }

    torch.save(ensemble_config, file_path)
    print(f"Ensemble configuration saved to {file_path}")

# Example usage
model_paths = ['best_model1.pt', 'best_model2.pt', 'best_model3.pt', 'best_model4.pt', 'best_model5.pt']
alphas = [alpha_1, alpha_2, alpha_3, alpha_4, alpha_5]
save_ensemble_config(model_paths, alphas)

Ensemble configuration saved to ensemble_model_config.pt


In [7]:
from ultralytics import YOLO
import torch

# Load the trained YOLO model
model = YOLO('best_model1.pt')

# Example weight for the model
weight = 0.5  # Adjust this weight based on your requirement

# Function to apply weight to confidence scores
def apply_weight_to_scores(results, weight):
    all_boxes = []
    all_scores = []
    all_classes = []
    
    for result in results:
        boxes = result.boxes
        all_boxes.append(boxes.xyxy)
        all_scores.append(boxes.conf * weight)  # Apply weight to confidence scores
        all_classes.append(boxes.cls)
    
    if all_boxes:
        # Concatenate all predictions
        boxes_tensor = torch.cat(all_boxes)
        scores_tensor = torch.cat(all_scores)
        classes_tensor = torch.cat(all_classes)

        # Perform NMS
        nms_indices = torch.ops.torchvision.nms(boxes_tensor, scores_tensor, 0.5)
        
        # Create final predictions
        final_boxes = boxes_tensor[nms_indices]
        final_scores = scores_tensor[nms_indices]
        final_classes = classes_tensor[nms_indices]
        
        return list(zip(final_boxes, final_scores, final_classes))
    
    return []

# Predict using the single model
image_path = 'javanicus.jpeg'
results = model.predict(image_path, conf=0.25)  # Adjust confidence threshold if needed

# Apply weights to the predictions
final_predictions = apply_weight_to_scores(results, weight)

# Print or visualize the final predictions
for i, (box, score, cls) in enumerate(final_predictions):
    print(f"Detection {i+1}:")
    print(f"  Box: {box.tolist()}")
    print(f"  Score: {score.item():.4f}")
    print(f"  Class: {cls.item()}")



image 1/1 /home/fnilvu/Documents/Project Medaka/API Ikan/javanicus.jpeg: 352x640 1 O.javanicus, 126.0ms
Speed: 50.8ms preprocess, 126.0ms inference, 64.4ms postprocess per image at shape (1, 3, 352, 640)
Detection 1:
  Box: [59.406219482421875, 59.04910659790039, 246.71923828125, 122.02272033691406]
  Score: 0.4339
  Class: 1.0


In [8]:
from ultralytics import YOLO
import torch

# Load the trained YOLO model
model = YOLO('ensemble_model_config.pt')

# Example weight for the model
weight = 0.5  # Adjust this weight based on your requirement

# Function to apply weight to confidence scores
def apply_weight_to_scores(results, weight):
    all_boxes = []
    all_scores = []
    all_classes = []
    
    for result in results:
        boxes = result.boxes
        all_boxes.append(boxes.xyxy)
        all_scores.append(boxes.conf * weight)  # Apply weight to confidence scores
        all_classes.append(boxes.cls)
    
    if all_boxes:
        # Concatenate all predictions
        boxes_tensor = torch.cat(all_boxes)
        scores_tensor = torch.cat(all_scores)
        classes_tensor = torch.cat(all_classes)

        # Perform NMS
        nms_indices = torch.ops.torchvision.nms(boxes_tensor, scores_tensor, 0.5)
        
        # Create final predictions
        final_boxes = boxes_tensor[nms_indices]
        final_scores = scores_tensor[nms_indices]
        final_classes = classes_tensor[nms_indices]
        
        return list(zip(final_boxes, final_scores, final_classes))
    
    return []

# Predict using the single model
image_path = 'javanicus.jpeg'
results = model.predict(image_path, conf=0.25)  # Adjust confidence threshold if needed

# Apply weights to the predictions
final_predictions = apply_weight_to_scores(results, weight)

# Print or visualize the final predictions
for i, (box, score, cls) in enumerate(final_predictions):
    print(f"Detection {i+1}:")
    print(f"  Box: {box.tolist()}")
    print(f"  Score: {score.item():.4f}")
    print(f"  Class: {cls.item()}")


KeyError: 'model'